## Interactive resistor pipeline
Use the widgets to select an image and tweak parameters for each stage of the pipeline. The figure updates in real time and shows the detected color bands and computed resistance.

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.signal import find_peaks

from resistor_reader.preprocess import auto_white_balance
from resistor_reader.roi import _foreground_mask, _largest_component, _remove_leads, _rotate_and_crop
from resistor_reader.bands import _classify, _TOLERANCE_COLORS
from resistor_reader.resolve import resolve_value

In [ ]:
def preprocess_step(array, top, bottom, left, right):
    top, bottom = sorted((top, bottom))
    left, right = sorted((left, right))
    cropped = array[top:bottom, left:right]
    processed = auto_white_balance(cropped)
    hsv = cv2.cvtColor(processed, cv2.COLOR_RGB2HSV)
    return processed, hsv

def roi_step(image, hsv, dist_thresh):
    mask = _foreground_mask(hsv)
    mask = _remove_leads(mask, dist_thresh=dist_thresh)
    mask = _largest_component(mask)
    crop, _ = _rotate_and_crop(image, mask)
    return crop, mask

def segment_step(image, peak_distance):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    col_means = lab.mean(axis=0)
    base = np.median(col_means, axis=0)
    dist = np.linalg.norm(col_means - base, axis=1)
    dist_smooth = cv2.GaussianBlur(dist[None, :], (1,9), 0).ravel()
    peaks, _ = find_peaks(dist_smooth, distance=max(1, image.shape[1] // peak_distance))
    if len(peaks) < 4:
        raise ValueError('unable to find four bands')
    peak_vals = dist_smooth[peaks]
    centers = np.sort(peaks[np.argsort(peak_vals)[-4:]])
    segments = []
    for c in centers:
        val = dist_smooth[c]
        left = c
        while left > 0 and dist_smooth[left] > 0.5 * val:
            left -= 1
        right = c
        w = dist_smooth.size - 1
        while right < w and dist_smooth[right] > 0.5 * val:
            right += 1
        segments.append((int(left), int(right)))
    segments.sort(key=lambda x: x[0])
    labels = [_classify(image[:, s:e]) for s, e in segments]
    flipped = False
    if labels and labels[0] in _TOLERANCE_COLORS and labels[-1] not in _TOLERANCE_COLORS:
        labels = labels[1:] + labels[:1]
        segments = segments[1:] + segments[:1]
        flipped = True
    overlay = cv2.resize(image, (600, 400), interpolation=cv2.INTER_NEAREST)
    h, w = image.shape[:2]
    scale_x, scale_y = 600 / w, 400 / h
    if flipped:
        overlay = cv2.flip(overlay, 1)
    for (s, e), lbl in zip(segments, labels):
        s_up = int(s * scale_x)
        e_up = int(e * scale_x)
        if flipped:
            s_up, e_up = 600 - e_up, 600 - s_up
        top_y, bottom_y = 0, int((h - 1) * scale_y)
        cv2.rectangle(overlay, (s_up, top_y), (e_up - 1, bottom_y), (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(overlay, lbl, (s_up + 2, int(15 * scale_y)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
    return labels, overlay

def run_pipeline(image_name, top, bottom, left, right, dist_thresh, peak_distance):
    array = np.asarray(Image.open(os.path.join('resistor_pictures', image_name)).convert('RGB'))
    proc, hsv = preprocess_step(array, top, bottom, left, right)
    roi, mask = roi_step(proc, hsv, dist_thresh)
    labels, overlay = segment_step(roi, peak_distance)
    value = resolve_value(labels)
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(array); axes[0].set_title('original'); axes[0].axis('off')
    axes[1].imshow(proc); axes[1].set_title('preprocess'); axes[1].axis('off')
    axes[2].imshow(roi); axes[2].set_title('roi'); axes[2].axis('off')
    axes[3].imshow(overlay); axes[3].set_title('bands'); axes[3].axis('off')
    fig.suptitle(f'{labels} -> {value:.2f} Ω')
    plt.show()

In [ ]:
image_options = sorted(f for f in os.listdir('resistor_pictures') if f.lower().endswith('.jpg'))
image_dropdown = widgets.Dropdown(options=image_options, description='image')
top_slider = widgets.IntSlider(value=64, min=0, max=479, description='top')
bottom_slider = widgets.IntSlider(value=480, min=1, max=480, description='bottom')
left_slider = widgets.IntSlider(value=36, min=0, max=639, description='left')
right_slider = widgets.IntSlider(value=598, min=1, max=640, description='right')
dist_slider = widgets.FloatSlider(value=15.0, min=1.0, max=30.0, step=0.5, description='lead_dist')
peak_slider = widgets.IntSlider(value=20, min=1, max=50, description='peak_dist')

ui = widgets.VBox([image_dropdown, top_slider, bottom_slider, left_slider, right_slider, dist_slider, peak_slider])
out = widgets.interactive_output(run_pipeline, {'image_name': image_dropdown,
                                                'top': top_slider,
                                                'bottom': bottom_slider,
                                                'left': left_slider,
                                                'right': right_slider,
                                                'dist_thresh': dist_slider,
                                                'peak_distance': peak_slider})
display(ui, out)